In [ ]:
import ollama
from nomic import embed
import sentence_transformers as SBERT
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from typing import Optional, List

OLLAMA_NOMIC_MODEL = 'nomic-embed-text'

def cosine_similarity(vec1: list[float], vec2: list[float]):
    """Compute cosine similarity between two vectors."""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def ollama_nomic_embed(prompt: str) -> list[float]:

    response = ollama.embeddings(
        model=OLLAMA_NOMIC_MODEL,
        prompt=prompt
    )

    return response['embedding']



def reduce_and_plot(vectors: np.ndarray, labels: Optional[List[str]] = None, title: str = '2D Embedding Projection') -> None:
    """Reduce to 2D using PCA and plot."""
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(vectors)
    
    plt.figure(figsize=(8,6))
    plt.scatter(reduced[:,0], reduced[:,1], c='skyblue', s=60)
    
    if labels:
        for i, label in enumerate(labels):
            plt.text(reduced[i,0], reduced[i,1], label, fontsize=9)

    plt.title(title)
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.grid(True)
    plt.show()



In [ ]:
### Creating semantic embeddings using nomic via ollama model
### Test cases ###

response = ollama.embeddings(
    model='nomic-embed-text',
    prompt='facebook/react'
)

print(response['embedding'])  # This is your vector!

response2 = ollama.embeddings(
    model='nomic-embed-text',
    prompt='vuejs/vue'
)

vec1: np.array = np.array(response['embedding'])
vec2: np.array = np.array(response2['embedding'])
vectors: np.array = np.array([vec1,vec2])

cosine_similarity(response['embedding'], response2['embedding'])





In [ ]:
from pathlib import Path
import csv

foss_proj_space_csv: Path = Path("../csv_github_data/FOSS_projects_space.csv")



def vectorize_foss_names(csv_data_file: Path, embed_model: str = 'ollama') -> list[list[str]]:
    foss_name_vectors: list[list[str]] = []
    with open(csv_data_file, mode="r",newline='') as file:

        reader = csv.reader(file)

        for row in reader:
            print(row[0])
            vectorized_name: list[float] = ollama_nomic_embed(row[0]) ### row[0] is jsut the first entry without the brackets

            foss_name_vectors.append(vectorized_name)
    return foss_name_vectors

foss_name_vectors_space: list[list[str]] = vectorize_foss_names(foss_proj_space_csv)

In [ ]:
### Below is the schema that I'm going to use for my data in the vector database
### I'm including the fields of the metadata that I want

# Embedding 1: Foss project name (with space)

'''
Foss project name (with a space)
UID: aka hash of project name (with a space)
foss project description


'''

# Embed 2: Foss project name (with space) + foss project description